In [45]:
from collections import defaultdict

import morfeusz2
from termcolor import colored

In [2]:
EXACT = [
    '../outputs/contents_exact_postinglists.txt',
    '../outputs/titles_exact_postinglists.txt',
]
LEMATIZED = [
    '../outputs/contents_lematized_postinglists.txt',
    '../outputs/titles_lematized_postinglists.txt'
]
morph = morfeusz2.Morfeusz()

In [3]:
def read_text(lines):
    content = []
    from_title = 0
    contents = []
    titles = []
    for line in lines:
        if "TITLE:" in line:
            from_title = 0
            title = line[7:]
            titles.append([title])
            if len(content) > 0:
                contents.append(content)
            content = []
        if len(line.strip())>0:
            content.append(line)
        from_title += 1
    if len(content) > 0:
        contents.append(content)
    return contents, titles

In [4]:
def find_word(word, paths):
    # find word in postinglists and return indexes of corresponding documents
    idxs = []
    for path in paths:
        with open(path, 'r') as file:
            idx = []
            for line in file:
                partitioned = line.partition(":")
                key = partitioned[0]
                if key == word:
                    idx = partitioned[2].split(", ")
                    idx = [int(w.strip()) for w in idx]
                    break
            idxs.append(idx)
    return idxs

In [5]:
def prepare_question(question):
    # split and lematize question
    lematized_question = []
    analysis = morph.analyse(question)
    question = question.split(" ")
    for i, j, interp in analysis:
        lematized_question.append(interp[1].partition(":")[0])
    return lematized_question, question

In [6]:
def prepare_hits(question):
    # return hits per word in question
    lematized_question, question = prepare_question(question)
    exact_hits = []
    lematized_hits = []
    for word in question:
        exact_hits.append(find_word(word, EXACT))
    for word in lematized_question:
        lematized_hits.append(find_word(word, LEMATIZED))
    return exact_hits, lematized_hits

In [7]:
def count_hits(hits):
    # count hits per document
    contents_counts = defaultdict(int)
    titles_counts = defaultdict(int)
    for current_word_hits in hits:
        for article_idx in current_word_hits[0]:
            contents_counts[article_idx] += 1
        for article_idx in current_word_hits[1]:
            titles_counts[article_idx] += 1
    return contents_counts, titles_counts

In [8]:
def prune_hits(hits):
    hits = {k: v for k, v in sorted(hits.items(), key=lambda item: -item[1])}
    print(hits)

In [9]:
def score_documents(question):
    exact_hits, lematized_hits = prepare_hits(question)
    exact_contents_hits, exact_titles_hits = count_hits(exact_hits)
    lematized_contents_hits, lematized_titles_hits = count_hits(lematized_hits)
    all_hits = set(exact_contents_hits.keys()) | set(exact_titles_hits.keys()) | set(lematized_contents_hits.keys()) | set(lematized_titles_hits.keys())
    scores = {}
    for hit in all_hits:
        scores[hit] = 1/hit
        if hit in exact_titles_hits:
            scores[hit] += exact_titles_hits[hit] * 10
        if hit in exact_contents_hits:
            scores[hit] += exact_contents_hits[hit] * 5
        if hit in lematized_titles_hits:
            scores[hit] += lematized_titles_hits[hit] * 3
        if hit in lematized_contents_hits:
            scores[hit] += lematized_contents_hits[hit] * 1
    scores = {k: v for k, v in sorted(scores.items(), key=lambda item: -item[1])}
    return scores

In [ ]:
def lematize_quote(quote):
        analysis = morph.analyse(quote)
        current_idx = 0
        current_stemmed_quote = []
        for i, j, interp in analysis:
            if i == current_idx:
                current_stemmed_quote.append(interp[1].partition(":")[0])
                current_idx += 1
        current_stemmed_quote = [word for word in current_stemmed_quote if word.isalpha()]
        current_stemmed_quote = [word for word in current_stemmed_quote if len(word) > 3]
        return set(current_stemmed_quote)

In [49]:
def explain(question, title, content):
    lematized_question, question = prepare_question(question)
    analysis = morph.analyse(content)
    current_idx = 0
    to_color_content = []
    for i, j, interp in analysis:
        if i == current_idx:
            lemat = interp[1].partition(":")[0]
            current_idx += 1
            if lemat in lematized_question:
                to_color_content.append(interp[0])
    analysis = morph.analyse(title)
    current_idx = 0
    to_color_title = []
    for i, j, interp in analysis:
        if i == current_idx:
            lemat = interp[1].partition(":")[0]
            current_idx += 1
            if lemat in lematized_question:
                to_color_title.append(interp[0])

    for w in title.split(" "):
        if w in set(to_color_title):
            print(colored(w + " ", 'green'), end ='')
        else:
            print(w + " ", end ='')
    for w in content.split(" "):
        if w in set(to_color_content):
            print(colored(w + " ", 'green'), end ='')
        else:
            print(w + " ", end ='')
    print('\n')

In [34]:
with open("../outputs/indexed_contents.txt", "r") as f:
    content_lines = f.readlines()
with open("../outputs/indexed_titles.txt", "r") as f:
    titles_lines = f.readlines()

In [35]:
question = "liczba zdelokalizowanych elektronów związek aromatyczny"

In [36]:
scores = score_documents(question)

In [22]:
scores

{49044: 23.000020389854008,
 14607: 19.000068460327242,
 46167: 19.000021660493424,
 56825: 19.000017597888252,
 462221: 19.000002163467258,
 529064: 19.000001890130495,
 658998: 19.00000151745529,
 704459: 19.000001419529028,
 813055: 19.000001229929097,
 1140379: 19.000000876901453,
 4394: 14.000227583067819,
 30772: 13.000032497075264,
 12584: 12.000079465988557,
 15608: 12.000064069707841,
 210381: 12.000004753280953,
 546319: 12.000001830432403,
 963: 9.00103842159917,
 15017: 9.000066591196644,
 38022: 9.000026300562833,
 343408: 9.000002911988073,
 49141: 8.000020349606235,
 56495: 8.000017700681475,
 68154: 8.000014672653109,
 258679: 8.000003865795058,
 379842: 8.000002632673585,
 474224: 8.00000210870812,
 617642: 8.00000161906088,
 830784: 8.000001203682306,
 936276: 8.000001068061128,
 1182563: 8.00000084562091,
 1188574: 8.000000841344335,
 10106: 7.000098951118147,
 12008: 7.00008327781479,
 17250: 7.000057971014493,
 24559: 7.000040718270288,
 28194: 7.000035468539406,
 

In [37]:
best_document = list(scores.keys())[0]

In [50]:
explain(question, titles_lines[best_document], content_lines[best_document])

Rozpuszczalnik aromatyczny 
 Rozpuszczalnik aromatyczny Rozpuszczalniki aromatyczne – rodzaj rozpuszczalników organicznych , w których wykorzystuje się ciekłe związki aromatyczne . Typowe rozpuszczalniki aromatyczne są zasadniczo niepolarne , ale ustrukturalizowane , tzn . między cząsteczkami rozpuszczalnika występują oddziaływania słabe , które nadają mu cechy rozpuszczalnika bardzo słabo polarnego . Ta cecha powoduje , że niektóre związki chemiczne nierozpuszczalne w zwykłych rozpuszczalnikach apolarnych rozpuszczają się w rozpuszczalnikach aromatycznych . Obecność heteroatomu w cząsteczce silnie zwiększa polarność związku ( np . względna przenikalność elektryczna benzenu wynosi 2,3 , a pirydyny 13,3 ) , zmieniając diametralnie jego właściwości 
 



In [51]:
explain(question, titles_lines[14607], content_lines[14607])

Emisja elektronów 
 Emisja elektronów Emisja elektronów – zjawisko fizyczne polegające na wyrzucaniu elektronów z przewodnika , półprzewodnika , a w wyjątkowych przypadkach również z izolatorów do otoczenia . Do wyrzucenia elektronu z ciała potrzebna jest energia do pokonania przyciągania ciała w ilości równej lub większej niż praca wyjścia . W zależności od źródła energii dostarczanej emitowanym elektronom rozróżniane są: 
 



In [52]:
explain(question, titles_lines[46167], content_lines[46167])

Porfiryny 
 Porfiryny Porfiryny to organiczne związki heterocykliczne składające się z czterech pierścieni pirolowych , połączonych mostkami metinowymi =CH- w makrocykliczny układ o bardzo wysokim stopniu sprzężenia występujących w nim wiązań podwójnych . Pod względem budowy cząsteczki wszystkie związki z tej grupy są pochodnymi najprostszej porfiryny zwanej porfiną . Porfiryny mają charakter aromatyczny , zawierają 18 zdelokalizowanych elektronów typu formula_1 i spełniają regułę Hückla . Intensywnie absorbują światło w zakresie widzialnym . Posiadają intensywną barwę zarówno jako ciała stałe , jak i w roztworach . Roztwór protoporfiryny IX ( ooporfiryny ) możliwej do wyizolowania ze skorupki jaja kurzego intensywnie fluoryzuje po wystawieniu na światło UV . 
 



In [65]:
question = "W którym państwie w latach 50-tych XX wieku wydana została pierwsza karta kredytowa?"

In [66]:
scores = score_documents(question)

In [67]:
scores

{24982: 19.00004002882075,
 53362: 16.00001873992729,
 66527: 16.000015031490975,
 213459: 16.000004684740396,
 215606: 16.000004638089848,
 591341: 16.00000169107165,
 704114: 16.000001420224564,
 2052: 15.000487329434698,
 2089: 15.000478697941599,
 2157: 15.000463606861382,
 2268: 15.000440917107584,
 2269: 15.000440722785369,
 9854: 15.000101481631825,
 10001: 15.000099990000999,
 14334: 15.000069764197015,
 31830: 15.000031416902294,
 32400: 15.00003086419753,
 34252: 15.000029195375452,
 39155: 15.000025539522412,
 39250: 15.000025477707007,
 46204: 15.000021643147779,
 47102: 15.000021230520996,
 47105: 15.000021229168878,
 47108: 15.000021227816932,
 53352: 15.000018743439796,
 53364: 15.000018739224945,
 58875: 15.000016985138004,
 61615: 15.000016229814168,
 66641: 15.000015005777223,
 75886: 15.000013177661229,
 129338: 15.000007731679785,
 209821: 15.000004765967182,
 212787: 15.000004699535216,
 214121: 15.000004670256537,
 223918: 15.00000446592056,
 224530: 15.0000044537

In [68]:
best_document = list(scores.keys())[0]

In [69]:
explain(question, titles_lines[best_document], content_lines[best_document])

Karta kredytowa 
 Karta kredytowa Karta kredytowa – karta płatnicza , której wydanie jest związane z przyznaniem limitu kredytowego przez wydawcę karty , np . bank . Operacje wykonane przez posiadacza karty rozliczane są w ramach przyznanego limitu . Zwykle do wydania karty kredytowej nie jest potrzebne posiadanie rachunku płatniczego u wydawcy . Okresowo ( co miesiąc ) wydawca karty ( np . bank ) przysyła posiadaczowi karty wyciąg ( zestawienie transakcji ) z dokonanych operacji wraz z informacjami dotyczącymi spłaty . Limit kredytowy związany z kartą jest zazwyczaj oprocentowany wyżej niż linie kredytowe przyznawane do rachunków bieżących/oszczędnościowo-rozliczeniowych . 
 

